### 3.1.3

In [2]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
h = np.dot(c, W)

print(W)
print(h)

[[-0.65654138 -0.41160825 -0.3422761 ]
 [-0.80093074 -0.52841011  1.90745166]
 [-1.87827557 -0.14467712  0.31769021]
 [-1.03130222  0.13943371  1.53334829]
 [-0.44317075  0.73433231 -0.36708476]
 [ 0.99687667 -0.60261322  2.08880101]
 [ 0.19834116  1.06372729 -0.30294942]]
[[-0.65654138 -0.41160825 -0.3422761 ]]


In [4]:
import sys
sys.path.append('..')
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[ 0.78364751  0.13150247 -0.79394904]]


### 3.2.1

In [5]:
# サンプルのコンテキストデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重みの初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# レイヤの生成
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 順伝播
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[-0.58207634  1.22435512  0.73545215  1.06744593  1.29078525  0.54054917
   0.8617067 ]]
